<a href="https://colab.research.google.com/github/DMahjoob/AvenuesConsultingProjects/blob/main/Avenues_Coffee_Chats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
filename = files.upload()

Saving Responses 1.csv to Responses 1.csv


In [ ]:
import pandas as pd
import io
# comment here as a tester
df = pd.read_csv(io.BytesIO(filename["Responses 1.csv"]), skiprows=0)
df = df.drop(['Timestamp'], axis=1)

In [ ]:
memberList = []
num_rows = len(df)
row_index = 0

while row_index < num_rows:
  memberList.append([', '.join(map(str, df.iloc[row_index]))])
  row_index += 1

print(memberList)

[['Eric Park, Watching movies/TV shows, Listening to music, Trying new food spots, Cookies and Cream, Coffee, Chocolate, Time Travel, LA, Early bird gets the worm, R&B, In a big city, Trying new restaurants, Winter, Uber Eats 🚗, In my own room, 3'], ['Matthew Hui, Listening to music, Exploring LA, Trying new food spots, Cookies and Cream, Vanilla, Caramel, Invisibility, LA, Early bird gets the worm, Indie, Somewhere on a beach, Trying new restaurants, Fall, Taco stand, Ministry of Coffee (MOC), 7'], ["Darius Mahjoob, Watching movies/TV shows, Playing sports/working out, Listening to music, Coffee, Mint Chocolate Chip, Chocolate, Mind Reading, LA, What's a bedtime?, Jazz, Somewhere on a beach, Exploring LA, Summer, Taco stand, Leavey Library, 7"], ["Jenny Liu, Playing sports/working out, Cooking/baking, Exploring LA, Cookies and Cream, Mint Chocolate Chip, Cookie Dough, Time Travel, NY, What's a bedtime?, Hip hop/Rap, Somewhere on a beach, Trying new restaurants, Spring, Taco stand, Min

In [36]:
from transformers import BertTokenizer, BertModel
import torch
from sklearn.metrics.pairwise import cosine_similarity

# Load the BERT tokenizer and model once at the beginning
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Initialize variables to store the best match
best_match = -1
taken_people = []
all_people = []

# Loop through each member in the list
for member in memberList:
    token1 = tokenizer.tokenize(str(member))

    match_list = {}
    all_people.append(member[0].split(',')[0])
    # Loop through each member in the list again to compare
    for member2 in memberList:
        if member == member2:
            continue  # Skip comparison with itself

        token2 = tokenizer.tokenize(str(member2))

        # Combine tokens with [CLS] and [SEP]
        tokens1 = ['[CLS]'] + token1 + ['[SEP]']
        tokens2 = ['[CLS]'] + token2 + ['[SEP]']

        # Convert tokens to input IDs
        input_ids1 = torch.tensor([tokenizer.convert_tokens_to_ids(tokens1)])
        input_ids2 = torch.tensor([tokenizer.convert_tokens_to_ids(tokens2)])

        # Obtain the BERT embeddings
        with torch.no_grad():
            outputs1 = model(input_ids1)
            outputs2 = model(input_ids2)
            embeddings1 = outputs1.last_hidden_state[:, 0, :]  # [CLS] token
            embeddings2 = outputs2.last_hidden_state[:, 0, :]  # [CLS] token

        # Calculate similarity
        similarity_score = cosine_similarity(embeddings1.numpy(), embeddings2.numpy())
        # print(f"Similarity Score between '{member[0][0:16]}' and '{member2[0][0:16]}':", similarity_score[0][0])

        # Update the best match if needed
        match_list[similarity_score[0][0]] = member2[0].split(',')[0]


    myKeys = list(match_list.keys())
    myKeys.sort(reverse=True)
    sorted_dict = {i: match_list[i] for i in myKeys}

    for x in sorted_dict:
        if sorted_dict[x] not in taken_people and member[0].split(',')[0] not in taken_people:
            taken_people.append(sorted_dict[x])
            taken_people.append(member[0].split(',')[0])
            print("Best Match:", member[0].split(',')[0], "and", sorted_dict[x], "with a similarity score of", x)
            break

# If there is an odd number of people, prints the name of the unmatched person
for name in all_people:
  if name not in taken_people:
    print(f"Unmatched: {name}")

Best Match: Eric Park and Christian Yen with a similarity score of 0.9560399
Best Match: Matthew Hui and Ahaana Naimpally with a similarity score of 0.95869416
Best Match: Darius Mahjoob and Max Krech with a similarity score of 0.95972073
Best Match: Jenny Liu and Margaret Luwena with a similarity score of 0.93181
Best Match: Debra Erdenemandakh and Thomas Bittencourt with a similarity score of 0.95041317
Best Match: Grace Lee and Valen Lu with a similarity score of 0.9440884
Best Match: Reagan Sutton and Emma Chen with a similarity score of 0.97267735
Unmatched: Ryan Bohn
